In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, auc, roc_curve
from sklearn.datasets import make_classification
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import np_utils
from keras.callbacks import Callback, EarlyStopping


    
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.metrics.auc(y_true, y_pred)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        print(v)
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)
    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

def auc_roc_ignorante(y_true, y_pred):
    # any tensorflow metric
    value, _ = tf.metrics.auc(y_true,y_pred)
    return value

In [ ]:
# generation a small dataset
N_all = 1000
N_tr = int(0.7 * N_all)
N_te = N_all - N_tr

np.random.seed(0)
x_dim = 50
y_dim = 30
X = np.random.rand(N_all,x_dim)
y = np.random.randint(0,2,size=(N_all-1,y_dim))
y_app = np.zeros(shape= (1,y_dim))
y_app[0] = 1
y = np.vstack((y,y_app))

X_train, X_valid = X[:N_tr, :], X[N_tr:, :]
y_train, y_valid = y[:N_tr, :], y[N_tr:, :]

In [ ]:
# model & train
model = Sequential()
model.add(Dense(y_dim,input_shape=(X.shape[1],)))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', auc_roc])

my_callbacks = [EarlyStopping(monitor='auc_roc', patience=300, verbose=1, mode='max')]

In [ ]:
model.fit(X, y,batch_size=7000, epochs=20, verbose=1,callbacks=my_callbacks)

# # or use independent valid set
# model.fit(X_train, y_train,
#           validation_data=(X_valid, y_valid),
#           batch_size=32, nb_epoch=5, verbose=1,
#           callbacks=my_callbacks)

In [ ]:
y_pred = model.predict(X_train)

In [ ]:
aucs = []
for i in range(y.shape[1]):
    fpr, tpr, thresholds = roc_curve(y_train[:,i], y_pred[:,i],)
    aucs.append(auc(fpr,tpr))
    print(aucs[i])
np.mean(aucs)

In [ ]:
roc_auc_score(y_train, y_pred)